In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .master("local[*]")\
    .appName("Assignmet")\
    .getOrCreate()

22/05/10 06:00:35 WARN Utils: Your hostname, MohammadFatha resolves to a loopback address: 127.0.1.1; using 192.168.1.10 instead (on interface wlp1s0)
22/05/10 06:00:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/10 06:00:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
review_df = spark.read.json('reviews_Electronics_5.json')
review_df.show(10)

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|0528881469|  [0, 0]|    5.0|We got this GPS f...| 06 2, 2013| AO94DHGC771SJ|             amazdnu|     Gotta have GPS!|    1370131200|
|0528881469|[12, 15]|    1.0|I'm a professiona...|11 25, 2010| AMO214LNFCEI4|     Amazon Customer|   Very Disappointed|    1290643200|
|0528881469|[43, 45]|    3.0|Well, what can I ...| 09 9, 2010|A3N7T0DY83Y4IG|       C. A. Freeman|      1st impression|    1283990400|
|0528881469| [9, 10]|    2.0|Not going to writ...|11 24, 2010|A1H8PY3QHMQQA0|Dave M. Shaw "mac...|Great grafics, PO...|    1290556800|
|0528881469|  [0, 0]|    1.0|I've had mine for...|09 29

In [12]:
from pyspark.sql.functions import *
from datetime import datetime
from pyspark.sql.types import *

In [13]:
func =  udf (lambda x: datetime.strptime(x, '%m %d, %Y'), DateType())

review_df = review_df.withColumn('new_col', date_format(func(col('reviewTime')), 'MM-dd-yyy'))
review_df = review_df.withColumnRenamed('new_col', 'NewreviewTime')
review_df.show(5)

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+-------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|NewreviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+-------------+
|0528881469|  [0, 0]|    5.0|We got this GPS f...| 06 2, 2013| AO94DHGC771SJ|             amazdnu|     Gotta have GPS!|    1370131200|   06-02-2013|
|0528881469|[12, 15]|    1.0|I'm a professiona...|11 25, 2010| AMO214LNFCEI4|     Amazon Customer|   Very Disappointed|    1290643200|   11-25-2010|
|0528881469|[43, 45]|    3.0|Well, what can I ...| 09 9, 2010|A3N7T0DY83Y4IG|       C. A. Freeman|      1st impression|    1283990400|   09-09-2010|
|0528881469| [9, 10]|    2.0|Not going to writ...|11 24, 2010|A1H8PY3QHMQQA0|Dave M. Shaw "mac...|Great gr

In [3]:

schema = StructType([\
    StructField("reviewID", StringType(), True),\
    StructField("lowrate", IntegerType(), True),\
    StructField("overall", FloatType(), True),\
    StructField('highrate',IntegerType(), True)])
# schema = ['','','','highrate']
rating_df = spark.read.format('csv').schema(schema).load('ratings_Electronics.csv')
rating_df.show(10)

+--------------+---------+-------+----------+
|      reviewID|  lowrate|overall|  highrate|
+--------------+---------+-------+----------+
| AKM1MP6P0OYPR|132793040|    5.0|1365811200|
|A2CX7LUOHB2NDG|321732944|    5.0|1341100800|
|A2NWSAGRHCP8N5|439886341|    1.0|1367193600|
|A2WNBOD3WNDNKT|439886341|    3.0|1374451200|
|A1GI0U4ZRJA8WN|439886341|    1.0|1334707200|
|A1QGNMC6O1VW39|511189877|    5.0|1397433600|
|A3J3BRHTDRFJ2G|511189877|    2.0|1397433600|
|A2TY0BTJOTENPG|511189877|    5.0|1395878400|
|A34ATBPOK6HCHY|511189877|    5.0|1395532800|
| A89DO69P0XZ27|511189877|    5.0|1395446400|
+--------------+---------+-------+----------+
only showing top 10 rows



In [14]:
df = review_df.join(rating_df,review_df['reviewerID']==rating_df['reviewID'])\
    .select(review_df['reviewText'],rating_df['lowrate'],rating_df['highrate'],review_df['NewreviewTime'])
df.show(10)

+--------------------+-------+----------+-------------+
|          reviewText|lowrate|  highrate|NewreviewTime|
+--------------------+-------+----------+-------------+
|So the screen its...|   null|1400457600|   05-19-2014|
|So the screen its...|   null|1400457600|   05-19-2014|
|So the screen its...|   null|1400457600|   05-19-2014|
|So the screen its...|   null|1400457600|   05-19-2014|
|So the screen its...|   null|1400457600|   05-19-2014|
|So the screen its...|   null|1400457600|   05-19-2014|
|I had a complicat...|   null|1400457600|   05-19-2014|
|I had a complicat...|   null|1400457600|   05-19-2014|
|I had a complicat...|   null|1400457600|   05-19-2014|
|I had a complicat...|   null|1400457600|   05-19-2014|
+--------------------+-------+----------+-------------+
only showing top 10 rows



In [15]:

df.filter(col("lowrate").isNotNull()).show()


+--------------------+----------+----------+-------------+
|          reviewText|   lowrate|  highrate|NewreviewTime|
+--------------------+----------+----------+-------------+
|VERY HAPPY, it lo...| 972683275|1332028800|   03-18-2012|
|Installed easy an...| 972683275|1395878400|   03-27-2014|
|I consider myself...| 972683275|1360713600|   02-13-2013|
|My nook tablet ha...|1400501776|1341187200|   07-02-2012|
|I purchased a Kin...|1400599997|1265760000|   02-10-2010|
|I purchased this ...|1400599997|1265760000|   10-21-2008|
|This is exactly w...| 972683275|1363305600|   03-15-2013|
|This is a pretty ...|1400532620|1306713600|   05-30-2011|
|UPDATE 6/3/10:  B...|1400599997|1275177600|   05-30-2010|
|All the haters do...|1400532620|1306713600|   03-02-2012|
|The unit itself w...|1400532620|1306713600|   07-16-2011|
|They claim this w...|1400532620|1306713600|   11-27-2010|
|Unit works as exp...|1400532620|1306713600|   01-19-2011|
|Recording on the ...|1400532620|1306713600|   12-08-201

In [16]:
df.write.parquet('Assignment1paraquetfile')